Cómo Funciona
Modelo LLM:

Utilizamos un modelo de Hugging Face (distilbert-base-uncased) configurado para tareas de clasificación de texto.
Este modelo devuelve un puntaje de similitud entre el texto procesado (consulta + texto objetivo).
División en Palabras:

La consulta se divide en palabras para que SHAP pueda evaluar la importancia de cada una.
predict_text:

Genera texto filtrado basado en las palabras activas y calcula un puntaje de similitud con el modelo LLM.
Referencia SHAP:

Una matriz aleatoria de referencia ayuda a SHAP a generar explicaciones al comparar diferentes combinaciones de palabras.
Visualización:

Se utiliza shap.force_plot para mostrar cómo cada palabra contribuye al puntaje de similitud.

In [1]:
import shap
import numpy as np
from transformers import pipeline


In [22]:
import shap
from sentence_transformers import SentenceTransformer
import numpy as np

# Modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Consulta del usuario
query = "¿Cómo ayuda la cúrcuma a la salud?"

# Dividir la consulta en palabras
words = query.split()

# Vector de referencia para SHAP (matriz con una palabra activa por fila)
reference = np.eye(len(words))  # Matriz identidad, activa una palabra a la vez

# Definir una función de predicción para SHAP
def predict_text(words_to_keep):
    """
    Genera un vector embedding para una consulta parcial.
    `words_to_keep` es un arreglo que indica qué palabras incluir.
    """
    # Convertir `words_to_keep` a un formato binario si no lo es
    words_to_keep = np.round(words_to_keep).astype(bool)
    print(words_to_keep)
    
    # Filtrar palabras basadas en `words_to_keep`
    filtered_query = " ".join([words[i] for i in range(len(words)) if words_to_keep[i].all()])
    
    # Si no hay palabras seleccionadas, retornar un vector de ceros
    if not filtered_query.strip():
        return np.zeros_like(model.encode([""])[0])
    
    # Retornar el embedding de la consulta filtrada
    return model.encode([filtered_query])[0]

# Crear el explicador SHAP
explainer = shap.KernelExplainer(predict_text, reference)

# # Calcular los valores SHAP para la consulta completa (todas las palabras activas)
# shap_values = explainer.shap_values(np.ones(len(words)))

# # Visualizar los valores SHAP
# shap.force_plot(explainer.expected_value, shap_values, words)


[[ True False False False False False False]
 [False  True False False False False False]
 [False False  True False False False False]
 [False False False  True False False False]
 [False False False False  True False False]
 [False False False False False  True False]
 [False False False False False False  True]]


ValueError: operands could not be broadcast together with shapes (384,) (7,) 